# Developing SQLAlchemy methods

### Methods to Build:

- Setup: Connecting to the Database
- Create a Table for Pump Data
- Insert Data into the Table
- Query Data from the Table
- Update Data in the Table
- Delete Data from the Table
- Use SQLAlchemy ORM to Define Models and Perform CRUD Operations

In [1]:
import logging
from database_generator.logging_configuration import setup_logging_for_this_script
setup_logging_for_this_script()
# Get the logger for this module
logger = logging.getLogger(__name__)


from database_generator.helpers import (
    get_config_path,
    load_and_process_params,
)

from database_generator.get_data import (
    generate_stable_toy_data,
    introduce_exponential_anomalies,
    simulate_broken_sensor,
)

from database_generator.evaluate import (
    overlaid_plots_with_plotly,
)

from database_generator.db_operations import(
    create_sql_alchemy_engine,
    get_last_timestamp,
    query_data_by_datetime,
    store_pandas_dataframe_into_postegre,
)

In [2]:
# get the path to the .json file from the environment

path_for_the_json_file = get_config_path()
path_for_the_json_file

config_dict = load_and_process_params(path_for_the_json_file)

start_date_for_the_toy_dataset = config_dict['start_date_for_the_toy_dataset']
number_of_rows_for_stable_toy_data = config_dict['number_of_rows_for_stable_toy_data']
seed_for_the_stable_dataset = config_dict['seed_for_the_stable_dataset']

# Example usage
df_stable = generate_stable_toy_data(number_of_rows=number_of_rows_for_stable_toy_data, start_date=start_date_for_the_toy_dataset, seed_for_random=42)

# Create an engine using environment variables or specified parameters
engine = create_sql_alchemy_engine(
    # user='my_user',
    # password='my_secrets',
    # host='localhost',
    # port=5432,
    # dbname='data_generator_v1'
)

# Store pandas df into postgre

store_pandas_dataframe_into_postegre(df=df_stable)


In [ ]:
df_stable.head(10)


In [ ]:
df_stable.tail()

# theoretical stuff

### The Engine in SQLAlchemy is a core object that represents the interface to the database. Here’s a breakdown of what the Engine does:

- Connection Pooling: The Engine manages a pool of database connections. When you execute a query, the Engine provides a connection from this pool, making it efficient to execute multiple queries without needing to establish a new connection each time.

- Database Dialect: The Engine is configured with a dialect that is specific to the type of database you're using (PostgreSQL in this case). This dialect translates SQLAlchemy commands into the appropriate SQL for your database system.

- Execution Context: The Engine provides the execution context for SQL queries. It takes SQL expressions and translates them into the SQL string that is sent to the database.

- Thread-Safe: The Engine is designed to be shared among multiple threads, and it's safe to use concurrently. This is especially useful for web applications where multiple requests need to interact with the database.

### Understanding the Syntax

### 1. with engine.connect() as connection:
Purpose: This line is using a context manager (with statement) to create a new database connection from the SQLAlchemy engine.

engine.connect():

This method is used to create a new Connection object. The Connection object represents an active database connection. It provides a way to execute SQL statements, manage transactions, and interact with the database.
When you call engine.connect(), SQLAlchemy establishes a connection to the database from the connection pool maintained by the engine.
with Statement (Context Manager):

The with statement ensures that resources are properly managed. When the code block inside the with statement is done executing, it automatically releases the connection back to the pool (or closes it if it's no longer needed). This prevents resource leaks and ensures efficient use of connections.
It also handles any exceptions that might occur within the block, ensuring the connection is properly closed even if an error occurs.
Benefit of Using with:

Automatic cleanup: You don’t need to manually close the connection. It’s done automatically when the block is exited, either after successful execution or an error.
Reduces boilerplate code: You don’t need to write explicit try/finally blocks to ensure cleanup.

### 2. connection.execute()
Purpose: This method is used to execute a SQL statement on the database.

How It Works:

connection.execute(...) takes an Executable object (like text()), a SQL expression, or a SQLAlchemy statement object (e.g., select(), insert(), update(), delete()), and sends it to the database for execution.
Why Use .execute()?:

It abstracts the complexity of sending SQL commands to the database, making it easier to work with different database backends (e.g., PostgreSQL, MySQL, SQLite) without needing to change your code.
.execute() is a powerful function that supports a wide range of SQLAlchemy constructs, making it versatile for both raw SQL execution and ORM-based queries.

### 3. .fetchone()
Purpose: This method fetches a single row from the result set of the executed SQL query.

How It Works:

When you execute a SQL query that returns data (like SELECT), the execute() method returns a Result object.
Calling .fetchone() on the Result object retrieves the next row of the result set as a tuple.
If there are no more rows available, .fetchone() returns None.
Why Use .fetchone()?:

Efficient Memory Usage: If you only need one row from the result set, .fetchone() is more memory-efficient than .fetchall(), which retrieves all rows at once.
Useful for Single Row Queries: If you know your query is designed to return only one row (e.g., SELECT 1), .fetchone() is appropriate.

### Summary
- engine.connect(): Establishes a connection to the database.
- with ... as ...:: A context manager to handle resource cleanup automatically.
- connection.execute(...): Executes a SQL statement or SQLAlchemy expression.
- fetchone(): Retrieves the next row from the result set of the executed SQL statement.

### 1. metadata = MetaData()

- MetaData in SQLAlchemy:

MetaData is a container object in SQLAlchemy that holds information about the database schema (i.e., tables, columns, constraints).
It acts as a central registry that stores all the schema constructs, such as tables, columns, and other schema elements.
When you create a MetaData object, you're essentially creating a blank registry that will hold the structure of your tables.
Purpose in the Function:

In the context of the create_table_from_dataframe function, metadata = MetaData() is used to define a new, empty metadata object where we can register our table definitions. This object will then be used to generate SQL commands to create the tables in the actual database.

### 2. table = Table(table_name, metadata, *columns)

- What is Table in SQLAlchemy?

Table is a SQLAlchemy class that represents a database table. It defines the table's name, the columns it contains, the data types for each column, and any other metadata such as primary keys and foreign keys.
How Table is Defined in the Code:

table = Table(table_name, metadata, *columns):
table_name: The name of the table you want to create in the database.
metadata: The MetaData instance where this table's schema will be registered.
*columns: A list of Column objects that define the structure of the table (e.g., column names and their types).
By defining the table this way, we are dynamically creating a schema based on the provided DataFrame.
Where table is Used:

The table object itself is not directly used later in the function. Instead, its definition is registered in the metadata object. When we call metadata.create_all(engine), it uses all the table definitions registered in metadata to create the tables in the database.

Why It Seems Unused:
Although it looks like table is not being used, it is indeed crucial for creating the schema. The table definition is stored in metadata when we define it with Table(table_name, metadata, *columns).

### 3. metadata.create_all(engine)

- How metadata.create_all(engine) Works:

metadata.create_all(engine) is a method that generates SQL CREATE TABLE statements for all the table objects registered with the MetaData instance (metadata) and executes them against the provided database engine.
The engine represents the connection to the database. When you call create_all(engine), SQLAlchemy translates the table definitions in metadata into the appropriate SQL commands for the specific database dialect (e.g., PostgreSQL, MySQL) and runs them to create the tables.
What Happens Under the Hood:

For each Table object registered in metadata, SQLAlchemy generates the SQL command for creating that table.
If you have multiple tables defined within metadata, it will create all of them in the database.
It also checks if the table already exists in the database. If it does, it will skip creating that table (unless specified otherwise).

### Summary of the Process
metadata = MetaData(): Creates a container to hold all the table definitions.
table = Table(table_name, metadata, *columns): Defines a table schema dynamically and registers it with the metadata object.
metadata.create_all(engine): Generates and executes the SQL commands to create all tables registered in metadata in the connected database.


### Explanation of Inspector Usage
The Inspector is a class in SQLAlchemy that provides a generalized interface to database schema information. It's a powerful tool for introspecting (i.e., examining) the schema of a database.

- How the Inspector Works:

1. What is Inspector:

Inspector is part of SQLAlchemy's sqlalchemy.engine.reflection module.

It provides methods to inspect database schema details such as tables, columns, indexes, constraints, etc.
Creating an Inspector Instance:

- inspector = inspect(engine):
inspect() is a function that returns an instance of the Inspector class for a given Engine or Connection.

The engine parameter is the SQLAlchemy Engine instance connected to the database.
When you call inspect(engine), SQLAlchemy constructs an Inspector object that provides methods to interact with the database schema.
Checking for Table Existence with Inspector:

- inspector.has_table(table_name):
This method checks if a table with the specified name (table_name) exists in the current database schema.
If the table exists, it returns True; otherwise, it returns False.
This is useful for ensuring that a table is not created if it already exists, preventing potential conflicts or errors.

2. Other Common Inspector Methods:

- get_table_names(): Returns a list of all table names in the current schema.
- get_columns(table_name): Returns a list of column names and their details for a specified table.
- get_primary_keys(table_name): Returns a list of primary key column names for a specified table.
- get_foreign_keys(table_name): Returns a list of foreign keys and their details for a specified table.
- get_indexes(table_name): Returns a list of indexes and their details for a specified table.

### Explanation:

1. Convert DataFrame to List of Dictionaries:
- df.to_dict(orient='records'): This converts the DataFrame into a list of dictionaries, where each dictionary represents a row of data. This is compatible with SQLAlchemy's bulk insert operations.
Prepare the Insert Statement:


2. insert(table): Creates an INSERT SQL statement for the specified table object.
Execute Bulk Insert:


3. connection.execute(insert_stmt, data_to_insert): Executes the INSERT statement using the connection object. This performs a bulk insert of all the rows in one operation.


4. Commit the Transaction:
- connection.commit(): Commits the transaction to save the changes to the database.

### Executing the Insert Statement:

- connection.execute(insert_stmt, data_to_insert) is the core of the operation. Here’s what happens under the hood:
SQLAlchemy Engine Prepares the SQL Statement: The insert_stmt object, along with data_to_insert, is handed over to SQLAlchemy's engine and connection objects.

- Generating Parameterized SQL: SQLAlchemy generates a parameterized SQL query. Parameterized queries are used to safely insert data, protecting against SQL injection and improving performance. For example:
sql

- Example:

INSERT INTO sensor_data (index_and_id, Temperature_C, Pressure_MPa, ...)

VALUES (:index_and_id_1, :Temperature_C_1, :Pressure_MPa_1, ...),
       (:index_and_id_2, :Temperature_C_2, :Pressure_MPa_2, ...),
       ...

The placeholders (:index_and_id_1, :Temperature_C_1, etc.) represent the parameterized query placeholders that will be replaced by the actual values from data_to_insert.

- Bulk Insertion:
SQLAlchemy automatically bundles the multiple rows from data_to_insert and inserts them in a single bulk operation.
This approach is more efficient than inserting rows one by one, as it reduces the number of database round-trips.
The actual database adapter (in this case, psycopg2 for PostgreSQL) handles the underlying bulk insertion logic, and SQLAlchemy provides a convenient interface for interacting with it.
Committing the Transaction:

- After executing the execute() method, you call connection.commit() to commit the transaction. This makes all the changes (i.e., inserted rows) persistent in the database.
If you were to omit commit(), the changes would not be saved to the database.